In [2]:
# https://pyimagesearch.com/2020/08/10/opencv-sudoku-solver-and-ocr/
# https://stackoverflow.com/questions/58669313/how-can-i-remove-a-specific-digit-from-the-mnist-dataset-imported-from-keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report


def remove(digit, data, labels):
    idx = (labels != digit).nonzero()
    return data[idx], labels[idx]

((x_train, y_train), (x_test, y_test)) = mnist.load_data()

inputShape = (28, 28, 1)

x_train,y_train = remove(0,x_train,y_train)
x_test,y_test = remove(0,x_test,y_test)

x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

le = LabelBinarizer()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)


11501568/11490434 [==============================] - 0s 0us/step


In [10]:
optimizer = Adam(learning_rate=1e-1)
EPOCHS = 20
BATCH_SIZE = 128

model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape,activation="relu"))
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape,activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same",input_shape=inputShape,activation="relu"))
model.add(Conv2D(64, (3, 3), padding="same",input_shape=inputShape,activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding="same",input_shape=inputShape,activation="relu"))
model.add(Conv2D(128, (3, 3), padding="same",input_shape=inputShape,activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(64, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(9, activation = "softmax"))



model.compile(loss="categorical_crossentropy", 
			optimizer=optimizer,metrics=["accuracy"])

model.fit(x_train, y_train,
		validation_data=(x_test, y_test),
		batch_size=BATCH_SIZE,
		epochs=EPOCHS,
		verbose=1)

predictions = model.predict(x_test)

print(classification_report(y_test.argmax(axis=1),predictions.argmax(axis=1),
							target_names=[str(x) for x in le.classes_]))


Epoch 1/20
423/423 [==============================] - 8s 16ms/step - loss: 0.6578 - accuracy: 0.7811 - val_loss: 1.1881 - val_accuracy: 0.7365
Epoch 2/20
423/423 [==============================] - 6s 15ms/step - loss: 0.2014 - accuracy: 0.9452 - val_loss: 0.2169 - val_accuracy: 0.9427
Epoch 3/20
423/423 [==============================] - 6s 14ms/step - loss: 0.1610 - accuracy: 0.9583 - val_loss: 0.1287 - val_accuracy: 0.9649
Epoch 4/20
423/423 [==============================] - 6s 14ms/step - loss: 0.1175 - accuracy: 0.9708 - val_loss: 0.0650 - val_accuracy: 0.9855
Epoch 5/20
423/423 [==============================] - 6s 14ms/step - loss: 0.1350 - accuracy: 0.9693 - val_loss: 0.0652 - val_accuracy: 0.9866
Epoch 6/20
423/423 [==============================] - 6s 14ms/step - loss: 0.0949 - accuracy: 0.9777 - val_loss: 0.0380 - val_accuracy: 0.9914
Epoch 7/20
423/423 [==============================] - 6s 14ms/step - loss: 0.0935 - accuracy: 0.9783 - val_loss: 0.0865 - val_accuracy: 0.9804

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
model.save('/content/drive/MyDrive/Colab Notebooks/OpenCV Sudoku Solver/keras_MNIST_trained.h5',save_traces='h5')